# Optimizing Model Parameters
相关参考链接  
[教程链接](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)
[3B1B关于back propagation的讲解](https://www.youtube.com/watch?v=tIeHLnjs5U8)

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters
* numer of epochs
* batch size
* learning rate

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

## Loss Function

In [4]:
loss_fn = nn.CrossEntropyLoss()

## Optimizer

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Full Implementation

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307389  [   64/60000]
loss: 2.297274  [ 6464/60000]
loss: 2.278791  [12864/60000]
loss: 2.277838  [19264/60000]
loss: 2.258614  [25664/60000]
loss: 2.231742  [32064/60000]
loss: 2.233716  [38464/60000]
loss: 2.198623  [44864/60000]
loss: 2.210703  [51264/60000]
loss: 2.178663  [57664/60000]
Test Error: 
 Accuracy: 37.9%, Avg loss: 2.171231 

Epoch 2
-------------------------------
loss: 2.181728  [   64/60000]
loss: 2.175353  [ 6464/60000]
loss: 2.120527  [12864/60000]
loss: 2.139343  [19264/60000]
loss: 2.089977  [25664/60000]
loss: 2.034651  [32064/60000]
loss: 2.058110  [38464/60000]
loss: 1.978789  [44864/60000]
loss: 1.995381  [51264/60000]
loss: 1.926898  [57664/60000]
Test Error: 
 Accuracy: 55.5%, Avg loss: 1.919141 

Epoch 3
-------------------------------
loss: 1.951653  [   64/60000]
loss: 1.926196  [ 6464/60000]
loss: 1.810832  [12864/60000]
loss: 1.850012  [19264/60000]
loss: 1.746130  [25664/60000]
loss: 1.691947  [32064/600